In [2]:
import pandas as pd

df1 = pd.read_csv(r"/content/drive/MyDrive/africa.csv")
df1.head(2)

,id,type,subcategories,name,description,rating,latitude,longitude,numberOfReviews,amenities,...,RankingType,country,city,regional_rating,image,webUrl,amenities_cleaned,amenities_str,combined_features,description_clean
0,7945044,ATTRACTION,Sights & Landmarks,Kuminda Farm,It's a small farm and we are into Argo tourism...,5.0,-20.98,27.25,22,bathroom only,...,things to do,Botswana,Francistown,3.00,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com/Attraction_Review-...,[],NaN,kuminda farm sights landmarks botswana francis...,it s a small farm and we are into argo tourism...
1,1743605,ATTRACTION,"Casinos & Gambling, Fun & Games",Gaborone Sun,Botswana is where the white-hot Kalahari Deser...,4.0,-24.65,25.93,124,bathroom only,...,things to do,Botswana,Gaborone,6.25,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com/Attraction_Review-...,[],NaN,gaborone sun casinos gambling fun games botswa...,botswana is where the white hot kalahari deser...


In [3]:
df2 = pd.read_csv(r"/content/drive/MyDrive/reviews_model_data.csv")
df2.head(2)

,userId,placeid,rating,lang,text
0,A87669AAD9DA05FFBD46F1334B329FFD,2189822,5,en,Gee is a passionate tour guide. The animals a...
1,9FFED7DDC68883BBB8F4024333970E9A,2189822,5,en,Lots to see. Easy to get to from the Safari Lo...


In [4]:
df1.columns

Index(['id', 'type', 'subcategories', 'name', 'description', 'rating',
       'latitude', 'longitude', 'numberOfReviews', 'amenities', 'LowerPrice',
       'UpperPrice', 'Rank', 'Total', 'Location', 'RankingType', 'country',
       'city', 'regional_rating', 'image', 'webUrl', 'amenities_cleaned',
       'amenities_str', 'combined_features', 'description_clean'],
      dtype='object')

In [5]:
df2.columns

Index(['userId', 'placeid', 'rating', 'lang', 'text'], dtype='object')

In [13]:
!pip install numpy==1.24.4 --force-reinstall


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 26.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
pymc 5.21.1 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
blosc2 3.2.0 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.


In [1]:
!pip install scikit-surprise


In [16]:
import numpy as np
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

df1 = pd.read_csv(r"/content/drive/MyDrive/africa.csv")
df2 = pd.read_csv(r"/content/drive/MyDrive/reviews_model_data.csv")

# Filter reviews to include only placeids that are present in africa_final_data
valid_place_ids = set(df1['id'].astype(str))  # Ensure string match
filtered_reviews = df2[df2['placeid'].astype(str).isin(valid_place_ids)]

print(f"Total original reviews: {len(df2)}")
print(f"Filtered reviews (matching with africa_final_data): {len(filtered_reviews)}")

# Define the rating reader
reader = Reader(rating_scale=(1, 5))

# Load dataset for Surprise
data = Dataset.load_from_df(filtered_reviews[['userId', 'placeid', 'rating']], reader)

# Split the data (80-20)
trainset, testset = train_test_split(data, test_size=0.10, random_state=42)

print("Filtered dataset loaded and split into training and testing sets.")



Total original reviews: 78572
Filtered reviews (matching with africa_final_data): 31079
Filtered dataset loaded and split into training and testing sets.


In [17]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, KNNBasic, KNNWithMeans, SVDpp
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")




# Auto-select relevance threshold
rating_min = df2['rating'].min()
rating_max = df2['rating'].max()
threshold = round((rating_min + rating_max) / 2, 2)
print(f"Auto-selected threshold: {threshold}")

# Surprise Reader and Dataset
reader = Reader(rating_scale=(rating_min, rating_max))


# Evaluation function
def precision_recall_at_k(predictions, k=5, threshold=threshold):
    user_est_true = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions, recalls = [], []
    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        top_k = user_ratings[:k]
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in top_k)
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in top_k)

        precision = n_rel_and_rec_k / n_rec_k if n_rec_k else 0
        recall = n_rel_and_rec_k / n_rel if n_rel else 0

        precisions.append(precision)
        recalls.append(recall)

    return np.mean(precisions), np.mean(recalls)


Auto-selected threshold: 3.0


In [18]:
print("="*40)
print("Training KNNBasic")
trainset1, testset1 = train_test_split(data, test_size=0.10, random_state=42)
model1 = KNNBasic(sim_options={'name': 'msd', 'user_based': False})
model1.fit(trainset1)
predictions1 = model1.test(testset1)

rmse1 = accuracy.rmse(predictions1, verbose=False)
mae1 = accuracy.mae(predictions1, verbose=False)
precision1, recall1 = precision_recall_at_k(predictions1, k=5, threshold=threshold)

print("KNNBasic Results:")
print(f"  RMSE        : {rmse1:.4f}")
print(f"  MAE         : {mae1:.4f}")
print(f"  Precision@5 : {precision1:.4f}")
print(f"  Recall@5    : {recall1:.4f}")

actual_ratings1 = [pred.r_ui for pred in predictions1]
predicted_ratings1 = [pred.est for pred in predictions1]


Training KNNBasic
Computing the msd similarity matrix...
Done computing similarity matrix.
KNNBasic Results:
  RMSE        : 0.7248
  MAE         : 0.4821
  Precision@5 : 0.9739
  Recall@5    : 0.9741


In [19]:
print("="*40)
print("Training KNNWithMeans")
trainset2, testset2 = train_test_split(data, test_size=0.10, random_state=42)
model2 = KNNWithMeans(sim_options={'name': 'msd', 'user_based': False})
model2.fit(trainset2)
predictions2 = model2.test(testset2)

rmse2 = accuracy.rmse(predictions2, verbose=False)
mae2 = accuracy.mae(predictions2, verbose=False)
precision2, recall2 = precision_recall_at_k(predictions2, k=5, threshold=threshold)

print("KNNWithMeans Results:")
print(f"  RMSE        : {rmse2:.4f}")
print(f"  MAE         : {mae2:.4f}")
print(f"  Precision@5 : {precision2:.4f}")
print(f"  Recall@5    : {recall2:.4f}")

actual_ratings2 = [pred.r_ui for pred in predictions2]
predicted_ratings2 = [pred.est for pred in predictions2]


Training KNNWithMeans
Computing the msd similarity matrix...
Done computing similarity matrix.
KNNWithMeans Results:
  RMSE        : 0.7204
  MAE         : 0.4817
  Precision@5 : 0.9733
  Recall@5    : 0.9734


In [20]:
print("="*40)
print("Training SVDpp")
trainset3, testset3 = train_test_split(data, test_size=0.10, random_state=42)
model3 = SVDpp()
model3.fit(trainset3)
predictions3 = model3.test(testset3)

rmse3 = accuracy.rmse(predictions3, verbose=False)
mae3 = accuracy.mae(predictions3, verbose=False)
precision3, recall3 = precision_recall_at_k(predictions3, k=5, threshold=threshold)

print("SVDpp Results:")
print(f"  RMSE        : {rmse3:.4f}")
print(f"  MAE         : {mae3:.4f}")
print(f"  Precision@5 : {precision3:.4f}")
print(f"  Recall@5    : {recall3:.4f}")

actual_ratings3 = [pred.r_ui for pred in predictions3]
predicted_ratings3 = [pred.est for pred in predictions3]


Training SVDpp
SVDpp Results:
  RMSE        : 0.6421
  MAE         : 0.3921
  Precision@5 : 0.9762
  Recall@5    : 0.9763


In [22]:
# Store predictions and names
prediction_sets = [
    (predictions1, "KNNBasic"),
    (predictions2, "KNNWithMeans"),
    (predictions3, "SVDpp")
]

# Display actual vs predicted ratings (limit 5 for each model)
for predictions, name in prediction_sets:
    actual = [pred.r_ui for pred in predictions]
    predicted = [pred.est for pred in predictions]
    print("\nResults for", name)
    print("Actual Ratings   :", actual[:50])
    print("Predicted Ratings:", predicted[:50])



Results for KNNBasic
Actual Ratings   : [4.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 3.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 2.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0]
Predicted Ratings: [4.692109685030925, 5, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 5, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 4.692109685030925, 5, 4.692109685030925, 4.692109685030925, 4.692109685030

K-FOLD CROSS VALIDATION

In [23]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from surprise import Dataset, Reader, KNNBasic, KNNWithMeans, SVDpp
from surprise.model_selection import cross_validate
import warnings
warnings.filterwarnings("ignore")

# Cross-validation evaluation function
def run_cross_validation(model, model_name, data, cv_folds=5):
    print("="*60)
    print(f"Evaluating RMSE of algorithm {model_name} on {cv_folds} split(s).\n")

    results = cross_validate(model, data, measures=['RMSE'], cv=cv_folds, verbose=False)

    rmse_scores = results['test_rmse']
    fit_times = results['fit_time']
    test_times = results['test_time']

    print(f"{'':20}Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std")
    print(f"RMSE (testset)    ", end="")
    print("  ".join(f"{score:.4f}" for score in rmse_scores), end="  ")
    print(f"{np.mean(rmse_scores):.4f}  {np.std(rmse_scores):.4f}")

    print(f"Fit time          ", end="")
    print("  ".join(f"{t:.2f}" for t in fit_times), end="  ")
    print(f"{np.mean(fit_times):.2f}  {np.std(fit_times):.2f}")

    print(f"Test time         ", end="")
    print("  ".join(f"{t:.2f}" for t in test_times), end="  ")
    print(f"{np.mean(test_times):.2f}  {np.std(test_times):.2f}")

    print("\nCross-Validation Results")
    print("RMSE Scores:", rmse_scores)
    print("Average RMSE:", np.mean(rmse_scores))
    print()

# Define baseline models
models = {
    "KNNBasic": KNNBasic(sim_options={'name': 'msd', 'user_based': False}),
    "KNNWithMeans": KNNWithMeans(sim_options={'name': 'msd', 'user_based': False}),
    "SVDpp": SVDpp()
}

# Run evaluation for all models
for name, model in models.items():
    run_cross_validation(model, name, data)


Evaluating RMSE of algorithm KNNBasic on 5 split(s).

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
                    Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std
RMSE (testset)    0.7124  0.7585  0.7089  0.7317  0.6929  0.7209  0.0225
Fit time          0.01  0.02  0.02  0.02  0.03  0.02  0.01
Test time         0.33  0.04  0.04  0.04  0.06  0.10  0.11

Cross-Validation Results
RMSE Scores: [0.71239946 0.75850422 0.70894593 0.73166952 0.692914  ]
Average RMSE: 0.7208866280421409

Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing sim

In [24]:
# Train the final SVDpp model on the full filtered dataset

from surprise import Dataset, Reader
from surprise import SVDpp

# Rebuild dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(filtered_reviews[['userId', 'placeid', 'rating']], reader)
trainset = data.build_full_trainset()

# Train the final model
final_model = SVDpp()
final_model.fit(trainset)


In [25]:
from surprise.model_selection import GridSearchCV
from surprise import SVDpp

# Define the parameter grid
param_grid = {
    'n_epochs': [10, 20, 30, 40, 50],          # More epochs = better learning
    'lr_all': [0.002, 0.005, 0.007, 0.01],     # Learning rate: try smaller steps
    'reg_all': [0.01, 0.02, 0.05, 0.1, 0.2]    # Regularization: control overfitting
}


# Use GridSearchCV for tuning
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=3, n_jobs=-1, joblib_verbose=1)
gs.fit(data)

# Best score and params
print("Best RMSE score from tuning:", gs.best_score['rmse'])
print("Best parameters:", gs.best_params['rmse'])

# Train with best parameters
best_model = gs.best_estimator['rmse']
best_model.fit(trainset)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.1min finished


Best RMSE score from tuning: 0.6506822885306679
Best parameters: {'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.1}


In [26]:
from surprise.model_selection import RandomizedSearchCV
from surprise import SVDpp

# Define the parameter distributions (same as GridSearchCV)
param_distributions = {
    'n_epochs': [10, 20, 30, 40, 50],
    'lr_all': [0.002, 0.005, 0.007, 0.01],
    'reg_all': [0.01, 0.02, 0.05, 0.1, 0.2]
}

# Use RandomizedSearchCV for tuning
rs = RandomizedSearchCV(SVDpp, param_distributions,
                        measures=['rmse'], cv=3,
                        n_iter=20,  # You can increase/decrease this
                        random_state=42,
                        n_jobs=-1, joblib_verbose=1)

# Fit on full data
rs.fit(data)

# Best score and params
print("Best RMSE score from tuning:", rs.best_score['rmse'])
print("Best parameters:", rs.best_params['rmse'])

# Train the best model on full training set
best_model = rs.best_estimator['rmse']
best_model.fit(trainset)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   27.5s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   35.1s finished


Best RMSE score from tuning: 0.6513926541067407
Best parameters: {'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.1}


In [27]:
from textblob import TextBlob
import pandas as pd

# Load reviews
reviews_df = pd.read_csv(r"/content/drive/MyDrive/reviews_model_data.csv")

# Define a function to calculate sentiment polarity
def get_sentiment(text):
    try:
        return TextBlob(str(text)).sentiment.polarity  # Range: -1 to 1
    except:
        return 0.0

# Apply sentiment analysis
reviews_df['sentiment_score'] = reviews_df['text'].apply(get_sentiment)

# Optional: Label the sentiment
def label_sentiment(score):
    if score > 0.1:
        return 'positive'
    elif score < -0.1:
        return 'negative'
    else:
        return 'neutral'

reviews_df['sentiment_label'] = reviews_df['sentiment_score'].apply(label_sentiment)

# Save for reuse
reviews_df.to_csv("reviews_with_sentiment.csv", index=False)


In [28]:
# Group by place to get average sentiment
place_sentiment = reviews_df.groupby('placeid')['sentiment_score'].mean().reset_index()
place_sentiment.columns = ['placeid', 'avg_sentiment_score']


In [ ]:
reviews_df

,userId,placeid,rating,lang,text,sentiment_score,sentiment_label
0,A87669AAD9DA05FFBD46F1334B329FFD,2189822,5,en,Gee is a passionate tour guide. The animals a...,0.300893,positive
1,9FFED7DDC68883BBB8F4024333970E9A,2189822,5,en,Lots to see. Easy to get to from the Safari Lo...,0.316667,positive
2,B5E56A483B579518DDD82A3DA0E94487,2189822,4,en,a kind of a zoo for injured and saved animals ...,0.286667,positive
3,882D0A6C7152105BB0D83C84F3CB160D,2189822,5,en,They do great rehabilitating injured animals. ...,0.125000,positive
4,203EBC7F3F51AAAA39A87D2E58842C76,2189822,5,en,"We took the guided tour from Isaac, who was gr...",0.292560,positive
...,...,...,...,...,...,...,...
78567,DE7F7643421284F6E26B31283D2CCB85,2720312,5,en,Nothing I disliked.\nStopped her for an excell...,0.266667,positive
78568,D9205755480636B049F9DAFB8BE6FF12,2720312,5,en,"For travellers to the south west of Uganda, th...",0.304167,positive
78569,FB3E9894020549D01D0468808AE93A5C,2720312,4,en,Located just a few miles outside Mbarara and h...,0.266242,positive
78570,8C31D999A4FC2AB2DC9824E3FFF82BE6,2720312,4,tr,Çok kaliteli ve temiz bir tesis . Yemekleri ço...,0.000000,neutral


In [29]:
unique_placeids = reviews_df['placeid'].unique()

# Display the unique placeid values
print("Unique placeids:", unique_placeids)

# Optional: Number of unique placeids
print("Total unique placeids:", len(unique_placeids))

Unique placeids: [ 2189822 12948270   553850 ...  4080840 10246297  2720312]
Total unique placeids: 1423


In [30]:
reviews_df.columns

Index(['userId', 'placeid', 'rating', 'lang', 'text', 'sentiment_score',
       'sentiment_label'],
      dtype='object')

In [34]:
import pandas as pd

# Load both datasets
africa_df = pd.read_csv("/content/drive/MyDrive/africa.csv")

# 1. Aggregate average sentiment score and average rating per placeid
agg_reviews = reviews_df.groupby('placeid').agg({
    'sentiment_score': 'mean',
    'rating': 'mean'
}).reset_index()

# Rename columns for clarity
agg_reviews.columns = ['id', 'avg_sentiment_score', 'avg_review_rating']  # 'placeid' renamed to 'id' for merge

# 2. Merge into africa_df using id
africa_df = pd.merge(africa_df, agg_reviews, on='id', how='left')

# 3. Fill NaNs if any (e.g., places without reviews)
africa_df['avg_sentiment_score'] = africa_df['avg_sentiment_score'].fillna(0.0)
africa_df['avg_review_rating'] = africa_df['avg_review_rating'].fillna(0.0)

# 4. Save final version (optional)
africa_df.to_csv("africa_data_with_sentiment.csv", index=False)

# Show a preview
africa_df[['id', 'name', 'avg_sentiment_score', 'avg_review_rating']].head()


,id,name,avg_sentiment_score,avg_review_rating
0,7945044,Kuminda Farm,0.330678,4.909091
1,1743605,Gaborone Sun,0.387416,4.020000
2,4162082,Pelican Lodge & Camping,0.000000,0.000000
3,1068835,Serowe Hotel,0.000000,0.000000
4,13477853,Leopard Plains,0.000000,0.000000


In [35]:
africa_df.head(5)

,id,type,subcategories,name,description,rating,latitude,longitude,numberOfReviews,amenities,...,city,regional_rating,image,webUrl,amenities_cleaned,amenities_str,combined_features,description_clean,avg_sentiment_score,avg_review_rating
0,7945044,ATTRACTION,Sights & Landmarks,Kuminda Farm,It's a small farm and we are into Argo tourism...,5.0,-20.98,27.25,22,bathroom only,...,Francistown,3.00,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com/Attraction_Review-...,[],NaN,kuminda farm sights landmarks botswana francis...,it s a small farm and we are into argo tourism...,0.330678,4.909091
1,1743605,ATTRACTION,"Casinos & Gambling, Fun & Games",Gaborone Sun,Botswana is where the white-hot Kalahari Deser...,4.0,-24.65,25.93,124,bathroom only,...,Gaborone,6.25,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com/Attraction_Review-...,[],NaN,gaborone sun casinos gambling fun games botswa...,botswana is where the white hot kalahari deser...,0.387416,4.020000
2,4162082,HOTEL,Specialty Lodging,Pelican Lodge & Camping,"The lodge is situated in Nata, Botswana. Clien...",3.0,-20.22,26.23,189,"['Pool', 'Internet', 'Free Internet', 'Free pa...",...,Nata,2.25,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com/Hotel_Review-g3161...,"['Pool', 'Internet', 'Free Internet', 'Free pa...","Pool, Internet, Free Internet, Free parking, R...",pelican lodge camping specialty lodging pool i...,the lodge is situated in nata botswana clients...,0.000000,0.000000
3,1068835,HOTEL,Bed and Breakfast,Serowe Hotel,"Set in beautiful gardens, the Serowe Hotel is ...",3.5,-22.41,26.72,79,"['Pool', 'Internet', 'Free parking', 'Restaura...",...,Serowe,7.00,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com/Hotel_Review-g4791...,"['Pool', 'Internet', 'Free parking', 'Restaura...","Pool, Internet, Free parking, Restaurant, Bar/...",serowe hotel bed and breakfast pool internet f...,set in beautiful gardens the serowe hotel is a...,0.000000,0.000000
4,13477853,HOTEL,Specialty Lodging,Leopard Plains,Experience in the middle of the bushveld a lux...,4.5,-20.18,24.17,56,"['Pool', 'Restaurant', 'Wifi', 'Breakfast incl...",...,Maun,8.10,https://media-cdn.tripadvisor.com/media/photo-...,https://www.tripadvisor.com/Hotel_Review-g3170...,"['Pool', 'Restaurant', 'Wifi', 'Breakfast incl...","Pool, Restaurant, Wifi, Breakfast included, In...",leopard plains specialty lodging pool restaura...,experience in the middle of the bushveld a lux...,0.000000,0.000000


In [36]:
africa_df.columns

Index(['id', 'type', 'subcategories', 'name', 'description', 'rating',
       'latitude', 'longitude', 'numberOfReviews', 'amenities', 'LowerPrice',
       'UpperPrice', 'Rank', 'Total', 'Location', 'RankingType', 'country',
       'city', 'regional_rating', 'image', 'webUrl', 'amenities_cleaned',
       'amenities_str', 'combined_features', 'description_clean',
       'avg_sentiment_score', 'avg_review_rating'],
      dtype='object')

In [38]:
import pandas as pd
import numpy as np
import re

# Load final data
df = pd.read_csv("africa_data_with_sentiment.csv")

# Fill NaN
df['avg_review_rating'] = df['avg_review_rating'].fillna(0)
df['avg_sentiment_score'] = df['avg_sentiment_score'].fillna(0)
df['rating'] = df['rating'].fillna(0)
df['amenities_str'] = df['amenities_str'].fillna("")

In [42]:
df.columns

Index(['id', 'type', 'subcategories', 'name', 'description', 'rating',
       'latitude', 'longitude', 'numberOfReviews', 'amenities', 'LowerPrice',
       'UpperPrice', 'Rank', 'Total', 'Location', 'RankingType', 'country',
       'city', 'regional_rating', 'amenities_cleaned', 'amenities_str',
       'combined_features', 'description_clean', 'avg_sentiment_score',
       'avg_review_rating'],
      dtype='object')

In [40]:
import pandas as pd

# Load your data
df = pd.read_csv("/content/drive/MyDrive/africa_final_data_with_sentiment.csv")

# Function to compute a score for sorting
def compute_final_score(row):
    sentiment = row['avg_sentiment_score']
    review_rating = row['avg_review_rating']
    base_rating = row['rating']
    if sentiment > 0 and review_rating > 0:
        return (sentiment * 0.6 + review_rating * 0.4)
    elif sentiment > 0:
        return sentiment
    elif review_rating > 0:
        return review_rating
    else:
        return base_rating

# Clean and prepare data
df['city'] = df['city'].astype(str).str.lower()
df['type'] = df['type'].astype(str).str.lower()
df['description_clean'] = df['description_clean'].astype(str).str.lower()
df['amenities_str'] = df['amenities_str'].astype(str).str.lower()
df['avg_sentiment_score'] = df['avg_sentiment_score'].fillna(0)
df['avg_review_rating'] = df['avg_review_rating'].fillna(0)
df['rating'] = df['rating'].fillna(0)
if 'country' in df.columns:
    df['country'] = df['country'].astype(str).str.lower()

# Ask user for input
user_input = input("🧭 Please describe your travel preferences (e.g., 'Suggest hotels in Gaborone with pool and restaurant, under low budget'):\n👉 ")

# Extract details from input
user_input = user_input.lower()
cities = df['city'].dropna().unique().tolist()
countries = df['country'].dropna().unique().tolist() if 'country' in df.columns else []

target_city = next((c for c in cities if c in user_input), "")
target_country = next((c for c in countries if c in user_input), "botswana")

low_budget = "low budget" in user_input or "cheap" in user_input or "affordable" in user_input
wants_hotels = "hotel" in user_input or "stay" in user_input or "lodge" in user_input
wants_attractions = "attraction" in user_input or "see" in user_input or "landmark" in user_input or "place" in user_input

all_amenities = ['pool', 'internet', 'wifi', 'restaurant', 'parking', 'bar', 'air conditioning']
desired_amenities = [a for a in all_amenities if a in user_input]

# Filter data based on city
city_df = df[df['city'].str.contains(target_city)] if target_city else df.copy()

# ------------------ Attractions ------------------
top_attractions = pd.DataFrame()
if wants_attractions:
    attractions_df = city_df[city_df['type'] == 'attraction']
    if attractions_df.empty:
        attractions_df = df[(df['type'] == 'attraction') & (df['country'].str.lower() == target_country)]
    attractions_df['final_score'] = attractions_df.apply(compute_final_score, axis=1)
    top_attractions = attractions_df.sort_values(by='final_score', ascending=False)[
        ['name', 'description', 'final_score', 'avg_sentiment_score', 'avg_review_rating']
    ].head(5)

# ------------------ Hotels ------------------
top_hotels = pd.DataFrame()
if wants_hotels:
    hotels_df = city_df[city_df['type'] == 'hotel']
    if desired_amenities:
        def amenities_match(amenities_str):
            return all(a in amenities_str for a in desired_amenities)
        hotels_df = hotels_df[hotels_df['amenities_str'].apply(amenities_match)]

    if low_budget and 'LowerPrice' in hotels_df.columns:
        hotels_df = hotels_df[hotels_df['LowerPrice'].notna()]
        hotels_df = hotels_df.sort_values(by='LowerPrice', ascending=True)

    hotels_df['final_score'] = hotels_df.apply(compute_final_score, axis=1)
    top_hotels = hotels_df.sort_values(by='final_score', ascending=False)[
        ['name', 'description', 'LowerPrice', 'amenities_str', 'final_score', 'avg_sentiment_score', 'avg_review_rating']
    ].head(5)

import textwrap

def print_attractions(df):
    print("🎯 TOP 5 ATTRACTIONS\n" + "="*70)
    if df.empty:
        print("🙁 No attractions found matching your criteria.\n")
    else:
        for idx, row in df.iterrows():
            print(f"\n🔹 {idx+1}. {row['name'].title()}")
            print("-" * 70)
            print("📜 Description:")
            print(textwrap.fill(row['description'], width=70))
            print(f"💬 Sentiment Score: {row['avg_sentiment_score']:.2f} | ⭐ Review Rating: {row['avg_review_rating']:.2f} | 🧮 Final Score: {row['final_score']:.2f}")
            print("="*70)

def print_hotels(df):
    print("🏨 TOP 5 HOTELS\n" + "="*70)
    if df.empty:
        print("🙁 No hotels found matching your criteria.\n")
    else:
        for idx, row in df.iterrows():
            print(f"\n🔹 {idx+1}. {row['name'].title()}")
            print("-" * 70)
            print("📜 Description:")
            print(textwrap.fill(row['description'], width=70))
            if pd.notna(row['LowerPrice']):
                print(f"💲 Lower Price: {row['LowerPrice']}")
            print(f"🛎️ Amenities: {row['amenities_str']}")
            print(f"💬 Sentiment Score: {row['avg_sentiment_score']:.2f} | ⭐ Review Rating: {row['avg_review_rating']:.2f} | 🧮 Final Score: {row['final_score']:.2f}")
            print("="*70)

# ------------------ Output ------------------
print("\n✅ Your preferences have been processed successfully!\n")

if wants_attractions:
    print_attractions(top_attractions)

if wants_hotels:
    print_hotels(top_hotels)

if not wants_hotels and not wants_attractions:
    print("❗ No specific hotel or attraction request detected. Try mentioning what you're looking for like 'hotels', 'attractions', or desired amenities.")



🧭 Please describe your travel preferences (e.g., 'Suggest hotels in Gaborone with pool and restaurant, under low budget'):
👉 BOtswana attractions

✅ Your preferences have been processed successfully!

🎯 TOP 5 ATTRACTIONS

🔹 954. Animalia
----------------------------------------------------------------------
📜 Description:
Center of information about life in nubia(nature and culture)it is an
ethno-biological museum which consists of three parts1-traditional
nubian house 2- museum shows fauna in
nubia(Animals,reptiles,birds,fish,insects all stuffed)rocks and
stones.3-the roof terrace where drinks,lunch,quick meals are
served.also lectures about nubia ,the nile and elephantine are given
over looking the gardens.Animalia also organise birdwatching tour in
the nile held by professional guides MOHAMMED SOBHY and his Daughter
Fatma.
💬 Sentiment Score: 0.00 | ⭐ Review Rating: 0.00 | 🧮 Final Score: 5.00

🔹 6142. Bateleur Helicopters
--------------------------------------------------------------

In [45]:
import pandas as pd
import textwrap

# Load your data
df = pd.read_csv("africa_data_with_sentiment.csv")

# Function to compute a score for sorting
def compute_final_score(row):
    sentiment = row['avg_sentiment_score']
    review_rating = row['avg_review_rating']
    base_rating = row['rating']
    if sentiment > 0 and review_rating > 0:
        return (sentiment * 0.6 + review_rating * 0.4)
    elif sentiment > 0:
        return sentiment
    elif review_rating > 0:
        return review_rating
    else:
        return base_rating

# Clean and prepare data
df['city'] = df['city'].astype(str).str.lower()
df['type'] = df['type'].astype(str).str.lower()
df['description_clean'] = df['description_clean'].astype(str).str.lower()
df['amenities_str'] = df['amenities_str'].astype(str).str.lower()
df['avg_sentiment_score'] = df['avg_sentiment_score'].fillna(0)
df['avg_review_rating'] = df['avg_review_rating'].fillna(0)
df['rating'] = df['rating'].fillna(0)
if 'country' in df.columns:
    df['country'] = df['country'].astype(str).str.lower()

# Ask user for input
user_input = input("🧭 Please describe your travel preferences (e.g., 'Suggest hotels in Gaborone with pool and restaurant, under low budget'):\n👉 ")

# Extract details from input
user_input = user_input.lower()
cities = df['city'].dropna().unique().tolist()
countries = df['country'].dropna().unique().tolist() if 'country' in df.columns else []

target_city = next((c for c in cities if c in user_input), "")
target_country = next((c for c in countries if c in user_input), "botswana")

low_budget = "low budget" in user_input or "cheap" in user_input or "affordable" in user_input
wants_hotels = "hotel" in user_input or "stay" in user_input or "lodge" in user_input
wants_attractions = "attraction" in user_input or "see" in user_input or "landmark" in user_input or "place" in user_input

all_amenities = ['pool', 'internet', 'wifi', 'restaurant', 'parking', 'bar', 'air conditioning']
desired_amenities = [a for a in all_amenities if a in user_input]

# Filter data based on city
city_df = df[df['city'].str.contains(target_city)] if target_city else df.copy()

# ------------------ Attractions ------------------
top_attractions = pd.DataFrame()
if wants_attractions:
    attractions_df = city_df[city_df['type'] == 'attraction']
    if attractions_df.empty:
        attractions_df = df[(df['type'] == 'attraction') & (df['country'].str.lower() == target_country)]
    attractions_df['final_score'] = attractions_df.apply(compute_final_score, axis=1)
    top_attractions = attractions_df.sort_values(by='final_score', ascending=False)[
        ['name', 'type', 'description', 'final_score', 'avg_sentiment_score', 'avg_review_rating', 'city', 'webUrl', 'image']
    ].head(5)

# ------------------ Hotels ------------------
top_hotels = pd.DataFrame()
if wants_hotels:
    hotels_df = city_df[city_df['type'] == 'hotel']
    if desired_amenities:
        def amenities_match(amenities_str):
            return all(a in amenities_str for a in desired_amenities)
        hotels_df = hotels_df[hotels_df['amenities_str'].apply(amenities_match)]

    if low_budget and 'LowerPrice' in hotels_df.columns:
        hotels_df = hotels_df[hotels_df['LowerPrice'].notna()]
        hotels_df = hotels_df.sort_values(by='LowerPrice', ascending=True)

    hotels_df['final_score'] = hotels_df.apply(compute_final_score, axis=1)
    top_hotels = hotels_df.sort_values(by='final_score', ascending=False)[
        ['name', 'type', 'description', 'LowerPrice', 'amenities_str', 'final_score', 'avg_sentiment_score', 'avg_review_rating', 'city', 'webUrl', 'image']
    ].head(5)

def print_attractions(df):
    print("🎯 TOP 5 ATTRACTIONS\n" + "="*70)
    if df.empty:
        print("🙁 No attractions found matching your criteria.\n")
    else:
        for idx, row in df.iterrows():
            print(f"\n🔹 {idx+1}. {row['name'].title()} ({row['type'].upper()})")
            print("-" * 70)
            print(f"📍 Location: {row['city'].title()}")
            print("📜 Description:")
            print(textwrap.fill(row['description'], width=70))

            # Display image URL if available
            if 'image' in row and pd.notna(row['image']):
                print(f"🖼️ Image: {row['image']}")

            # Display web URL if available
            if 'webUrl' in row and pd.notna(row['webUrl']):
                print(f"🌐 Website: {row['webUrl']}")

            print(f"💬 Sentiment Score: {row['avg_sentiment_score']:.2f} | ⭐ Review Rating: {row['avg_review_rating']:.2f} | 🧮 Final Score: {row['final_score']:.2f}")
            print("="*70)

def print_hotels(df):
    print("🏨 TOP 5 HOTELS\n" + "="*70)
    if df.empty:
        print("🙁 No hotels found matching your criteria.\n")
    else:
        for idx, row in df.iterrows():
            print(f"\n🔹 {idx+1}. {row['name'].title()} ({row['type'].upper()})")
            print("-" * 70)
            print(f"📍 Location: {row['city'].title()}")
            print("📜 Description:")
            print(textwrap.fill(row['description'], width=70))

            # Display price if available
            if pd.notna(row['LowerPrice']):
                print(f"💲 Lower Price: {row['LowerPrice']}")

            # Display amenities
            print(f"🛎️ Amenities: {row['amenities_str']}")

            # Display image URL if available
            if 'image' in row and pd.notna(row['image']):
                print(f"🖼️ Image: {row['image']}")

            # Display web URL if available
            if 'webUrl' in row and pd.notna(row['webUrl']):
                print(f"🌐 Website: {row['webUrl']}")

            print(f"💬 Sentiment Score: {row['avg_sentiment_score']:.2f} | ⭐ Review Rating: {row['avg_review_rating']:.2f} | 🧮 Final Score: {row['final_score']:.2f}")
            print("="*70)

# ------------------ Output ------------------
print("\n✅ Your preferences have been processed successfully!\n")

if wants_attractions:
    print_attractions(top_attractions)

if wants_hotels:
    print_hotels(top_hotels)

if not wants_hotels and not wants_attractions:
    print("❗ No specific hotel or attraction request detected. Try mentioning what you're looking for like 'hotels', 'attractions', or desired amenities.")

# Function to recommend places based on preferences
def recommend_places(df, num_recommendations=5):
    print("\n🌟 RECOMMENDED PLACES BASED ON RATINGS\n" + "="*70)

    if df.empty:
        print("🙁 No recommendations available based on your criteria.\n")
        return

    # First prioritize places with good sentiment and review ratings
    recommendations = df[(df['avg_sentiment_score'] > 0) & (df['avg_review_rating'] > 0)].copy()
    recommendations['final_score'] = recommendations.apply(compute_final_score, axis=1)

    # If not enough recommendations, add places with only sentiment scores
    if len(recommendations) < num_recommendations:
        sentiment_only = df[(df['avg_sentiment_score'] > 0) & (df['avg_review_rating'] == 0)].copy()
        sentiment_only['final_score'] = sentiment_only['avg_sentiment_score']
        recommendations = pd.concat([recommendations, sentiment_only])

    # If still not enough, add places with only review ratings
    if len(recommendations) < num_recommendations:
        review_only = df[(df['avg_sentiment_score'] == 0) & (df['avg_review_rating'] > 0)].copy()
        review_only['final_score'] = review_only['avg_review_rating']
        recommendations = pd.concat([recommendations, review_only])

    # If still not enough, add places with only base ratings
    if len(recommendations) < num_recommendations:
        base_only = df[(df['avg_sentiment_score'] == 0) & (df['avg_review_rating'] == 0) & (df['rating'] > 0)].copy()
        base_only['final_score'] = base_only['rating']
        recommendations = pd.concat([recommendations, base_only])

    # Sort by final score and get top recommendations
    recommendations = recommendations.sort_values(by='final_score', ascending=False).head(num_recommendations)

    # Display recommendations
    for idx, row in recommendations.iterrows():
        print(f"\n🔹 {idx+1}. {row['name'].title()} ({row['type'].upper()})")
        print("-" * 70)
        print(f"📍 Location: {row['city'].title()}")

        # Display image URL if available
        if 'image' in row and pd.notna(row['image']):
            print(f"🖼️ Image: {row['image']}")

        # Display web URL if available
        if 'webUrl' in row and pd.notna(row['webUrl']):
            print(f"🌐 Website: {row['webUrl']}")

        # Display scores based on what's available
        print("📊 Ratings:")
        if row['avg_sentiment_score'] > 0:
            print(f"  💬 Sentiment Score: {row['avg_sentiment_score']:.2f}")
        if row['avg_review_rating'] > 0:
            print(f"  ⭐ Review Rating: {row['avg_review_rating']:.2f}")
        if row['avg_sentiment_score'] == 0 and row['avg_review_rating'] == 0:
            print(f"  ⭐ Base Rating: {row['rating']:.2f}")
        print(f"  🧮 Final Score: {row['final_score']:.2f}")
        print("="*70)

# Generate general recommendations regardless of user input
print("\n🔍 Would you like to see general recommendations? (y/n)")
show_recommendations = input("👉 ")
if show_recommendations.lower() in ['y', 'yes']:
    recommend_places(df)

🧭 Please describe your travel preferences (e.g., 'Suggest hotels in Gaborone with pool and restaurant, under low budget'):
👉 i want to go to  attractions in Gaborone

✅ Your preferences have been processed successfully!

🎯 TOP 5 ATTRACTIONS

🔹 150. Kalahari Culture & Nature Safaris (ATTRACTION)
----------------------------------------------------------------------
📍 Location: Gaborone
📜 Description:
Kalahari Culture & Nature Safaris (KCNS) not just offers the best
safari experience in Botswana, but also shows you the best of
Botswana's heritage and culture. We show you what others wont! We
believe that a wildlife Safari in Botswana is incomplete without the
people. History, culture & heritage coupled with nature based safaris
is the foundation of KCNS.  -Heritage and cultural tours all over
Botswana -Safaris all over Botswana -City tours -Self drive tour guide
and trip organize
🖼️ Image: https://media-cdn.tripadvisor.com/media/photo-o/1d/1b/ef/c4/caption.jpg
🌐 Website: https://www.trip